In [1]:
import numpy as np
import pandas as pd
import QSTK.qstkutil.qsdateutil as du
import QSTK.qstkutil.tsutil as tsu
import QSTK.qstkutil.DataAccess as da

import datetime as dt
import matplotlib.pyplot as plt


In [2]:
ls_symbols = ["AAPL", "GLD", "GOOG", "XOM"]
dt_start = dt.datetime(2011, 1, 1)
dt_end = dt.datetime(2011, 12, 31)
dt_timeofday = dt.timedelta(hours=16) # time duration
ldt_timestamps = du.getNYSEdays(dt_start, dt_end, dt_timeofday)  # the days that NYSE is open
datetimes = [d.to_pydatetime() for d in ldt_timestamps]

c_dataobj = da.DataAccess('Yahoo')
ls_keys = ['open', 'high', 'low', 'close', 'volume', 'actual_close']
ldf_data = c_dataobj.get_data(ldt_timestamps, ls_symbols, ls_keys)
d_data = dict(zip(ls_keys, ldf_data))

In [3]:
def simulate(startdate,enddate,symbols,allocations):
    dt_timeofday = dt.timedelta(hours=16) # time duration
    ldt_timestamps = du.getNYSEdays(dt_start, dt_end, dt_timeofday)  # the days that NYSE is open
    datetimes = [d.to_pydatetime() for d in ldt_timestamps]
    c_dataobj = da.DataAccess('Yahoo')
    ls_keys = ['open', 'high', 'low', 'close', 'volume', 'actual_close']
    ldf_data = c_dataobj.get_data(ldt_timestamps, ls_symbols, ls_keys)
    d_data = dict(zip(ls_keys, ldf_data))
    df_close = d_data['close'].copy().fillna(method='ffill')
    df_close = df_close.fillna(method='bfill')
    na_price = df_close.values
    allocation = [0.4,0.4,0.0,0.2]
    na_rets = tsu.returnize0(na_price)
    na_cum_rets = np.cumprod(1+na_rets,axis=0)
    port_cum_rets = np.sum(na_cum_rets * allocation,axis=1)
    port_daily_rets = tsu.returnize0(port_cum_rets.copy())
    vol = np.std(port_daily_rets)
    avg_daily_ret = np.mean(port_daily_rets)
    sharpe = avg_daily_ret / vol * np.sqrt(252)
    cum_return = port_cum_rets[-1]
    return sharpe,vol,avg_daily_ret,cum_return

In [5]:
allocation = [0.2,0.2,0.4,0.2]
sharpe,vol,avg_daily_ret,cum_ret = simulate(dt_start,dt_end,ls_keys,allocation)
print 'sharpe ratio = ', sharpe
print 'vol = ', vol
print 'daily average return = ', avg_daily_ret
print 'cum_ret = ', cum_ret - 1

sharpe ratio =  1.02828403099
vol =  0.0101467067654
daily average return =  0.000657261102001
cum_ret =  0.164872619645
